In [1]:
import numpy
import numba

In [2]:
@numba.jit(nopython=True)
def make_list():
    return [1, 2, 3.0]

print(make_list())

[1.0, 2.0, 3.0]


In [3]:
@numba.jit(nopython=True)
def return_first(a_list):
    return a_list[0]

def return_first_py(a_list):
    return a_list[0]

In [4]:
long_list = [0] * 1000
nb_result = %timeit -o return_first(long_list)
py_result = %timeit -o return_first_py(long_list)

nb_result.best / py_result.best

The slowest run took 4125.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 18.8 µs per loop
The slowest run took 6.85 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 158 ns per loop


119.12829532275568

In [5]:
long_array = numpy.array(long_list)
nb_result = %timeit -o return_first(long_array)
py_result = %timeit -o return_first_py(long_array)

nb_result.best / py_result.best

The slowest run took 81451.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 357 ns per loop
The slowest run took 135.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 223 ns per loop


1.6050316349021772

In [6]:
@numba.jit(nopython=True)
def filter_bad_try1(values, lower, upper):
    good = []
    for v in values:
        if lower < v < upper:
            good.append(v)
    return numpy.array(good)

In [7]:
a = numpy.random.uniform(-2000, 2000, int(1e7))

In [8]:
%timeit filter_bad_try1(a, -1000, 1000)

1 loop, best of 3: 123 ms per loop


In [9]:
@numba.jit(nopython=True)
def filter_bad_try2(values, lower, upper):
    good = numpy.empty_like(values)
    next_index = 0
    for v in values:
        if lower < v < upper:
            good[next_index] = v
            next_index += 1
    return good[:next_index]

In [10]:
%timeit filter_bad_try2(a, -1000, 1000)

10 loops, best of 3: 60.4 ms per loop


In [11]:
filter_bad_try2(a, -1000, 1000)

array([ 997.9944078 ,  687.29342808, -105.7498187 , ...,  905.98062605,
         -1.43046327,  -55.9636759 ])

In [12]:
@numba.jit(nopython=True)
def filter_bad_try3(values, lower, upper):
    good = numpy.empty_like(values)
    next_index = 0
    for v in values:
        if lower < v < upper:
            good[next_index] = v
            next_index += 1
    return good[:next_index].copy()

In [13]:
%timeit filter_bad_try3(a, -1000, 1000)

10 loops, best of 3: 73.6 ms per loop


In [14]:
def filter_bad_try4(values, lower, upper):
    return values[(lower < values) & (values < upper)]

In [15]:
%timeit filter_bad_try4(a, -1000, 1000)

10 loops, best of 3: 95.9 ms per loop


In [16]:
@numba.jit(nopython=True)
def filter_bad_try5(values, lower, upper):
    return values[(lower < values) & (values < upper)]

In [17]:
%timeit filter_bad_try5(a, -1000, 1000)

1 loop, best of 3: 59.7 ms per loop
